### Кросс-внимание (Cross-Attention)

Кросс-внимание — это очень простая концепция, такая же, как обычное внимание, но $x$ в $Q = W_q(x)$ и $x$ в $W_k(x)/W_v(x)$ теперь принадлежат разным последовательностям. "Внимание", которое запросы (`queries`) уделяют ключам и значениям (`keys/values`), теперь работает **между** последовательностями, а не внутри одной последовательности токенов.  

На самом деле, это "оригинальное" внимание — именно так оно было представлено в знаменитой статье ["Attention is All You Need"](https://arxiv.org/abs/1706.03762), где механизм внимания изначально разрабатывался для задачи машинного перевода. Там кросс-внимание позволяло любому токену в фразе `"I am a guy"` обращать внимание на любой токен в `"Je suis un mec"`, что, очевидно, очень полезно для перевода.  

В современных языковых моделях (LLM) кросс-внимание **почти не используется** (или не используется вовсе). Однако оно остаётся фундаментальным инструментом во многих других современных подходах:  

- **RAG** — для внимания к топ-k эмбеддингам, полученным от модели поиска/реранкера,  
- **Мультимодальные LLM** — для взаимодействия токенов разных модальностей,  
- **Условные диффузионные модели** — чтобы направлять латентное пространство или делать его функцией от внешней последовательности (контекста).  

Это **очень мощный приём**, который стоит хорошо понимать, если вы планируете работать с архитектурами нейросетей.  

#### Интерпретация кросс-внимания  

Результирующий вектор:  

$$  
\text{Output} = \frac{1}{\sqrt{D}} \cdot \text{Softmax}(QK^T) V  
$$  

— это линейная комбинация векторов значений (`V`), но теперь они приходят **из другой последовательности**, а их веса (коэффициенты) выбираются **исходной последовательностью**.  

#### Личное замечание  

Это важный инструмент, если вы экспериментируете с архитектурами. Например, несколько лет назад я пытался улучшить перплексию языковой модели, заставив её обращать внимание на эмбеддинги, сгенерированные **визуальным энкодером**. Идея была в том, что если модель "увидит" изображения разных видов цветов, то сможет лучше различать их по текстовым описаниям.  

В итоге это **не сработало**, но позже появились похожие идеи, где кросс-внимание используется для улучшения LLM за счёт навыков других моделей (см. [Augmenting LLMs with Knowledge](https://arxiv.org/pdf/2401.02412)).  

In [ ]:
"""
Этот модуль реализует два варианта механизма внимания Multi-Head Attention (MHA)
с использованием библиотеки PyTorch:
1.  MHSA (Multi-Head Self-Attention): Стандартный механизм самовнимания, где
    запросы (queries), ключи (keys) и значения (values) генерируются из одного
    и того же входного тензора.
2.  CrossMHSA (Cross Multi-Head Self-Attention): Механизм перекрестного внимания,
    где запросы генерируются из одного входного тензора (x1), а ключи и значения -
    из другого (x2). Это позволяет модели соотносить информацию между двумя
    различными последовательностями.

Модуль также включает тестовый блок, который:
-   Сравнивает поведение MHSA (в режиме без каузальной маски) и CrossMHSA,
    когда на вход подаются идентичные последовательности.
-   Демонстрирует ключевую особенность CrossMHSA: способность обрабатывать
    входные последовательности (x1 и x2) разной длины, что невозможноЦ
    для стандартного MHSA.
-   Проверяет, что CrossMHSA выдает различные результаты при обработке
    одинаковых и разных последовательностей.

Этот код служит наглядным примером реализации и принципов работы
перекрестного внимания (Cross-Attention).
"""

# Стандартные библиотеки
import math
from typing import Tuple # Используется для аннотации типов кортежей

# Сторонние библиотеки
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
# Определение класса для Multi-Head Self-Attention (MHSA)
class MHSA(nn.Module):
    """
    Description:
    ---------------
        Реализует механизм Multi-Head Self-Attention (MHSA).
        В этом механизме запросы (queries), ключи (keys) и значения (values)
        вычисляются из одного и того же входного тензора.
        Может работать в каузальном режиме (для декодеров трансформера).

    Args:
    ---------------
        head_dim (int): Размерность каждого "внимания" (головы).
                        По умолчанию 64.
        d_model (int): Размерность входного и выходного тензора признаков
                       (размерность остаточного потока). По умолчанию 512.
        causal (bool): Если True, применяется каузальная маска, чтобы каждая
                       позиция могла обращать внимание только на предыдущие
                       позиции и на саму себя. По умолчанию True.

    Raises:
    ---------------
        AssertionError: Если `d_model` не делится нацело на `head_dim`.

    Examples:
    ---------------
        >>> mhsa_layer = MHSA(head_dim=64, d_model=512, causal=True)
        >>> input_tensor = torch.randn(2, 10, 512) # (batch, seq_len, d_model)
        >>> output_tensor = mhsa_layer(input_tensor)
        >>> print(output_tensor.shape)
        torch.Size([2, 10, 512])
    """
    def __init__(self, head_dim: int = 64, d_model: int = 512, causal: bool = True):
        super().__init__()
        self.head_dim: int = head_dim
        self.d_model: int = d_model
        # Проверка, что размерность модели делится на размерность головы
        # Это необходимо для корректного разделения на n_heads
        assert d_model % head_dim == 0, (
            "Ошибка: размерность головы (head_dim) не делит нацело "
            "размерность остаточного потока (d_model)."
        )
        self.n_heads: int = d_model // head_dim
        self.causal: bool = causal

        # Линейный слой для проекции входа в объединенное пространство Q, K, V
        # Размерность выхода 3 * d_model, так как Q, K, V имеют размерность d_model
        self.qkv_proj: nn.Linear = nn.Linear(d_model, 3 * d_model)
        # Линейный слой для финальной проекции результата внимания
        self.out_proj: nn.Linear = nn.Linear(d_model, d_model)

    # Прямой проход для MHSA
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Description:
        ---------------
            Выполняет прямой проход механизма Multi-Head Self-Attention.

        Args:
        ---------------
            x (torch.Tensor): Входной тензор с формой (B, S, D), где:
                              B - размер пакета (batch size)
                              S - длина последовательности (sequence length)
                              D - размерность признаков (d_model)

        Returns:
        ---------------
            torch.Tensor: Выходной тензор с той же формой (B, S, D),
                          представляющий собой результат применения внимания.

        Raises:
        ---------------
            RuntimeError: Может возникнуть при несоответствии размерностей
                          тензоров во время операций PyTorch.
        """
        # B - batch_size, S - sequence_length
        batch_size, seq_len, _ = x.shape

        # 1. Проекция входа на Q, K, V
        # x: [B, S, D] -> qkv: [B, S, 3*D]
        qkv: torch.Tensor = self.qkv_proj(x)

        # 2. Разделение на Q, K, V и на n_heads
        # qkv: [B, S, 3*D] -> [B, S, 3, n_heads, head_dim]
        qkv = qkv.reshape(
            batch_size, seq_len, 3, self.n_heads, self.head_dim
        )
        # Изменение порядка измерений для удобства матричных операций
        # qkv: [B, S, 3, n_heads, head_dim] -> [3, B, n_heads, S, head_dim]
        qkv = qkv.permute(2, 0, 3, 1, 4)
        # Разделение на отдельные тензоры Q, K, V
        # Каждый из q, k, v имеет форму: [B, n_heads, S, head_dim]
        q, k, v = qkv[0], qkv[1], qkv[2]

        # 3. Вычисление логитов внимания (матрица схожести)
        # Используется операция Эйнштейна для пакетного матричного умножения
        # q: [B, n_heads, S, head_dim], k: [B, n_heads, S, head_dim]
        # attn_logits: [B, n_heads, S, S]
        attn_logits: torch.Tensor = torch.einsum('bnid,bnjd->bnij', q, k)

        # 4. Нормализация логитов внимания
        # Деление на корень из head_dim для стабилизации градиентов
        normalize_factor: float = math.sqrt(self.head_dim)
        attn_logits = attn_logits / normalize_factor # Поэлементное деление

        # 5. Применение каузальной маски (если требуется)
        # Каузальная маска не позволяет позициям "смотреть вперед"
        if self.causal:
            # Создание маски: True для разрешенных позиций, False для запрещенных
            # mask: [S, S]
            mask: torch.Tensor = torch.arange(seq_len, device=x.device)[:, None] >= \
                                 torch.arange(seq_len, device=x.device)
            # Применение маски: запрещенные позиции заменяются на -inf
            # Это приведет к нулевым весам внимания после softmax
            attn_logits = torch.where(
                mask, attn_logits, float('-inf') * torch.ones_like(attn_logits)
            )

        # 6. Применение Softmax для получения весов внимания
        # A (Attention weights): [B, n_heads, S, S]
        attention_weights: torch.Tensor = F.softmax(attn_logits, dim=-1)

        # 7. Взвешенное суммирование значений (V) с использованием весов внимания
        # A: [B, n_heads, S, S], v: [B, n_heads, S, head_dim]
        # out: [B, n_heads, S, head_dim]
        out: torch.Tensor = torch.einsum('bnij,bnjd->bnid', attention_weights, v)

        # 8. Конкатенация выходов всех голов и изменение формы
        # out: [B, n_heads, S, head_dim] -> [B, S, n_heads, head_dim]
        out = out.transpose(1, 2)
        # out: [B, S, n_heads, head_dim] -> [B, S, D] (где D = n_heads * head_dim)
        out = out.reshape(batch_size, seq_len, -1)

        # 9. Финальная линейная проекция
        # out_proj: [D, D] @ out: [B, S, D] -> [B, S, D]
        return self.out_proj(out)

In [3]:
# Определение класса для Cross Multi-Head Attention (CrossMHSA)
class CrossMHSA(nn.Module):
    """
    Description:
    ---------------
        Реализует механизм Cross Multi-Head Attention (CrossMHSA).
        В этом механизме запросы (queries) генерируются из первого входного
        тензора (x1), а ключи (keys) и значения (values) - из второго
        входного тензора (x2). Это позволяет модели соотносить информацию
        между двумя различными последовательностями, которые могут иметь
        разную длину.

    Args:
    ---------------
        head_dim (int): Размерность каждого "внимания" (головы).
                        По умолчанию 64.
        d_model (int): Размерность входного и выходного тензора признаков
                       (размерность остаточного потока). По умолчанию 512.

    Raises:
    ---------------
        AssertionError: Если `d_model` не делится нацело на `head_dim`.

    Examples:
    ---------------
        >>> cross_mhsa_layer = CrossMHSA(head_dim=64, d_model=512)
        >>> x1_tensor = torch.randn(2, 10, 512) # (batch, seq_len1, d_model)
        >>> x2_tensor = torch.randn(2, 15, 512) # (batch, seq_len2, d_model)
        >>> output_tensor = cross_mhsa_layer(x1_tensor, x2_tensor)
        >>> print(output_tensor.shape) # Output shape matches x1's seq_len
        torch.Size([2, 10, 512])
    """
    def __init__(self, head_dim: int = 64, d_model: int = 512):
        super().__init__()
        self.head_dim: int = head_dim
        self.d_model: int = d_model
        # Проверка, что размерность модели делится на размерность головы
        assert d_model % head_dim == 0, (
            "Ошибка: размерность головы (head_dim) не делит нацело "
            "размерность остаточного потока (d_model)."
        )
        self.n_heads: int = d_model // head_dim

        # Отдельные линейные слои для проекции Q (из x1), K (из x2), V (из x2)
        self.q_proj: nn.Linear = nn.Linear(d_model, d_model)
        self.k_proj: nn.Linear = nn.Linear(d_model, d_model)
        self.v_proj: nn.Linear = nn.Linear(d_model, d_model)
        # Линейный слой для финальной проекции результата внимания
        self.out_proj: nn.Linear = nn.Linear(d_model, d_model)

    # Прямой проход для CrossMHSA
    def forward(self, x1: torch.Tensor, x2: torch.Tensor) -> torch.Tensor:
        """
        Description:
        ---------------
            Выполняет прямой проход механизма Cross Multi-Head Attention.
            Запросы (Q) генерируются из `x1`.
            Ключи (K) и значения (V) генерируются из `x2`.

        Args:
        ---------------
            x1 (torch.Tensor): Входной тензор для генерации запросов (Q).
                               Форма (B, S1, D), где:
                               B - размер пакета (batch size)
                               S1 - длина последовательности для запросов
                               D - размерность признаков (d_model)
            x2 (torch.Tensor): Входной тензор для генерации ключей (K) и
                               значений (V). Форма (B, S2, D), где:
                               B - размер пакета (batch size)
                               S2 - длина последовательности для ключей/значений
                               D - размерность признаков (d_model)

        Returns:
        ---------------
            torch.Tensor: Выходной тензор с формой (B, S1, D),
                          представляющий собой результат применения
                          перекрестного внимания. Длина последовательности
                          соответствует `x1`.

        Raises:
        ---------------
            RuntimeError: Может возникнуть при несоответствии размерностей
                          тензоров во время операций PyTorch.
        """
        # B - batch_size, S1 - sequence_length_x1, S2 - sequence_length_x2
        batch_size, seq_len1, _ = x1.shape
        _, seq_len2, _ = x2.shape

        # 1. Проекция входов на Q (из x1), K (из x2), V (из x2)
        # q: [B, S1, D]
        q: torch.Tensor = self.q_proj(x1)
        # k: [B, S2, D], v: [B, S2, D]
        k: torch.Tensor = self.k_proj(x2)
        v: torch.Tensor = self.v_proj(x2)

        # 2. Разделение Q, K, V на n_heads и изменение формы
        # q: [B, S1, D] -> [B, S1, n_heads, head_dim] -> [B, n_heads, S1, head_dim]
        q = q.reshape(
            batch_size, seq_len1, self.n_heads, self.head_dim
        ).transpose(1, 2)
        # k: [B, S2, D] -> [B, S2, n_heads, head_dim] -> [B, n_heads, S2, head_dim]
        k = k.reshape(
            batch_size, seq_len2, self.n_heads, self.head_dim
        ).transpose(1, 2)
        # v: [B, S2, D] -> [B, S2, n_heads, head_dim] -> [B, n_heads, S2, head_dim]
        v = v.reshape(
            batch_size, seq_len2, self.n_heads, self.head_dim
        ).transpose(1, 2)

        # 3. Вычисление логитов внимания (матрица схожести)
        # q: [B, n_heads, S1, head_dim], k: [B, n_heads, S2, head_dim]
        # attn_logits: [B, n_heads, S1, S2]
        # Каждая позиция из x1 (S1) обращает внимание на все позиции из x2 (S2)
        attn_logits: torch.Tensor = torch.einsum('bnid,bnjd->bnij', q, k)

        # 4. Нормализация логитов внимания
        normalize_factor: float = math.sqrt(self.head_dim)
        attn_logits = attn_logits / normalize_factor

        # 5. Применение Softmax для получения весов внимания
        # A (Attention weights): [B, n_heads, S1, S2]
        # Softmax применяется по последнему измерению (S2), т.е. по ключам из x2
        attention_weights: torch.Tensor = F.softmax(attn_logits, dim=-1)

        # 6. Взвешенное суммирование значений (V) с использованием весов внимания
        # A: [B, n_heads, S1, S2], v: [B, n_heads, S2, head_dim]
        # out: [B, n_heads, S1, head_dim]
        # Результат имеет длину последовательности S1 (от x1)
        out: torch.Tensor = torch.einsum('bnij,bnjd->bnid', attention_weights, v)

        # 7. Конкатенация выходов всех голов и изменение формы
        # out: [B, n_heads, S1, head_dim] -> [B, S1, n_heads, head_dim]
        out = out.transpose(1, 2)
        # out: [B, S1, n_heads, head_dim] -> [B, S1, D]
        out = out.reshape(batch_size, seq_len1, -1)

        # 8. Финальная линейная проекция
        # out_proj: [D, D] @ out: [B, S1, D] -> [B, S1, D]
        return self.out_proj(out)

In [4]:
# --- Тестовый блок ---
# Этот блок демонстрирует работу MHSA и CrossMHSA,
# а также их ключевые различия.

# Параметры для тестов
BATCH_SIZE: int = 2
SEQ_LEN: int = 5
DIM: int = 512
HEAD_DIM: int = 64

# --- Тест 1: Сравнение MHSA и CrossMHSA при одинаковых входных последовательностях ---
# Цель: Убедиться, что CrossMHSA(x, x) эквивалентен MHSA(x, causal=False)
# при идентичных весах.
print("--- Тест 1: Сравнение MHSA и CrossMHSA (одинаковые входы) ---")
# Создание случайного входного тензора
x_test_identical: torch.Tensor = torch.randn(BATCH_SIZE, SEQ_LEN, DIM)

# Инициализация модулей внимания
# Для справедливого сравнения, MHSA должен быть без каузальной маски
mhsa_module: MHSA = MHSA(head_dim=HEAD_DIM, d_model=DIM, causal=False)
cross_attn_module: CrossMHSA = CrossMHSA(head_dim=HEAD_DIM, d_model=DIM)

# Копирование весов из CrossMHSA в MHSA для эквивалентности
# Это необходимо, чтобы оба модуля выполняли идентичные вычисления
# при одинаковых входах.
with torch.no_grad(): # Отключаем вычисление градиентов для операций с весами
    # Копирование весов проекций Q, K, V
    # qkv_proj в MHSA объединяет проекции Q, K, V, поэтому конкатенируем
    # веса и смещения из q_proj, k_proj, v_proj из CrossMHSA.
    mhsa_module.qkv_proj.weight.copy_(torch.cat([
        cross_attn_module.q_proj.weight,
        cross_attn_module.k_proj.weight,
        cross_attn_module.v_proj.weight
    ], dim=0))
    mhsa_module.qkv_proj.bias.copy_(torch.cat([
        cross_attn_module.q_proj.bias,
        cross_attn_module.k_proj.bias,
        cross_attn_module.v_proj.bias
    ], dim=0))

    # Копирование весов выходной проекции
    cross_attn_module.out_proj.weight.copy_(mhsa_module.out_proj.weight)
    cross_attn_module.out_proj.bias.copy_(mhsa_module.out_proj.bias)

# Прямой проход через оба модуля
mhsa_output: torch.Tensor = mhsa_module(x_test_identical)
# Для CrossMHSA подаем один и тот же тензор x_test_identical в качестве x1 и x2
cross_attn_output_same_input: torch.Tensor = cross_attn_module(
    x_test_identical, x_test_identical
)

# Проверка, что выходы практически идентичны (с учетом погрешностей float)
# rtol и atol - относительная и абсолютная погрешности
are_outputs_close: bool = torch.allclose(
    mhsa_output, cross_attn_output_same_input, rtol=1e-4, atol=1e-4
)
print(
    f"При идентичных последовательностях и весах: выходы MHSA и "
    f"CrossMHSA(x,x) совпадают = {are_outputs_close}"
)


# --- Тест 2: Демонстрация работы CrossMHSA с последовательностями разной длины ---
# Цель: Показать, что CrossMHSA корректно обрабатывает x1 и x2 разной длины,
# и что выходная размерность по длине последовательности соответствует x1.
print("\n--- Тест 2: CrossMHSA с последовательностями разной длины ---")
SEQ_LEN_1: int = 5  # Длина первой последовательности (для Q)
SEQ_LEN_2: int = 7  # Длина второй последовательности (для K, V)

# Создание случайных входных тензоров разной длины
x1_diff_len: torch.Tensor = torch.randn(BATCH_SIZE, SEQ_LEN_1, DIM)
x2_diff_len: torch.Tensor = torch.randn(BATCH_SIZE, SEQ_LEN_2, DIM)

# Прямой проход CrossMHSA с разными последовательностями
# Веса cross_attn_module уже настроены из предыдущего теста
cross_attn_output_diff_len: torch.Tensor = cross_attn_module(
    x1_diff_len, x2_diff_len
)

# Проверка формы выходного тензора
# Ожидаемая форма: (BATCH_SIZE, SEQ_LEN_1, DIM), так как Q из x1
expected_shape: Tuple[int, int, int] = (BATCH_SIZE, SEQ_LEN_1, DIM)
assert cross_attn_output_diff_len.shape == expected_shape, (
    f"Ожидаемая форма {expected_shape}, "
    f"получено {cross_attn_output_diff_len.shape}"
)
print(
    f"CrossMHSA успешно обработал последовательности разной длины "
    f"({SEQ_LEN_1} и {SEQ_LEN_2})."
)
print(f"Форма выхода: {cross_attn_output_diff_len.shape} (соответствует x1).")

# Попытка использовать MHSA с разными длинами (неявная демонстрация)
# Стандартный MHSA ожидает один вход, из которого генерируются Q, K, V.
# Попытка передать ему две разные последовательности напрямую невозможна
# без модификации самого MHSA или входных данных (например, конкатенации
# и маскирования, что усложняет механизм).
# Cross-attention специально разработан для таких сценариев.
try:
    # Это не сработает напрямую с MHSA, так как он ожидает один вход.
    # Мы не будем вызывать ошибку, а просто констатируем факт.
    print(
        "Cross-attention позволяет использовать последовательности разной длины "
        "для Q и K/V, в то время как стандартный self-attention "
        "требует идентичные последовательности (или одну и ту же)."
    )
except Exception as e:
    # Этот блок не должен выполниться, так как мы не вызываем ошибку.
    print(f"Как и ожидалось, self-attention не может напрямую "
          f"обработать последовательности разной длины: {e}")


# --- Тест 3: Проверка, что CrossMHSA дает разные результаты для разных входов ---
# Цель: Убедиться, что результат CrossMHSA(x1, x2) отличается от
# CrossMHSA(x1, x1), если x2 != x1.
print("\n--- Тест 3: CrossMHSA дает разные результаты для разных входов ---")
# Используем x1_diff_len из предыдущего теста
x1_copy_for_test3: torch.Tensor = x1_diff_len.clone()

# Выход CrossMHSA, когда оба входа одинаковы (x1_copy_for_test3, x1_copy_for_test3)
cross_attn_output_same_again: torch.Tensor = cross_attn_module(
    x1_copy_for_test3, x1_copy_for_test3
)

# Выход CrossMHSA, когда входы разные (x1_diff_len, x2_diff_len)
# Этот результат (cross_attn_output_diff_len) уже вычислен в Тесте 2.

# Проверка, что результаты отличаются
are_results_different: bool = not torch.allclose(
    cross_attn_output_same_again,
    cross_attn_output_diff_len,
    rtol=1e-4,
    atol=1e-4
)
print(
    f"CrossMHSA(x1, x1) и CrossMHSA(x1, x2) (где x1 != x2) "
    f"дают разные результаты: {are_results_different}"
)

print("\nВсе тесты завершены!")

--- Тест 1: Сравнение MHSA и CrossMHSA (одинаковые входы) ---
При идентичных последовательностях и весах: выходы MHSA и CrossMHSA(x,x) совпадают = True

--- Тест 2: CrossMHSA с последовательностями разной длины ---
CrossMHSA успешно обработал последовательности разной длины (5 и 7).
Форма выхода: torch.Size([2, 5, 512]) (соответствует x1).
Cross-attention позволяет использовать последовательности разной длины для Q и K/V, в то время как стандартный self-attention требует идентичные последовательности (или одну и ту же).

--- Тест 3: CrossMHSA дает разные результаты для разных входов ---
CrossMHSA(x1, x1) и CrossMHSA(x1, x2) (где x1 != x2) дают разные результаты: True

Все тесты завершены!
